* The temporal order is scrambled in the test data making TS useless there.
* Still, just for learning / realism, we can still do it in the training data! 
* Let's add pseudo dates, and aggregate features on column subsets. Finally i'll run a model to predict the target!

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from datetime import datetime
from scipy.special import logsumexp

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report

In [ ]:
train = pd.read_csv("/kaggle/input/caltech-cs155-2020/train.csv")
test = pd.read_csv("/kaggle/input/caltech-cs155-2020/test.csv")
df = pd.concat([train,test],sort=False)
print(df.shape)
print(df.columns)
df.tail()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
## y is binary.
display(train["y"].describe())

In [ ]:
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

* Additional features could include: rank, which bid number is the max/min, etc' 
* features between the aggregated features (e.g. max bid div max ask..)

In [ ]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
    df[f"{group}_logsumexp"] = df[group_cols[group]].apply(logsumexp)
    
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    
df["last_price_div__mid"] = df["last_price"].div(df["mid"])

Additional features

In [ ]:
df["bid1sum"]=df["bid1"]*df["bid1vol"]
df["bid2sum"]=df["bid2"]*df["bid2vol"]
df["bid3sum"]=df["bid3"]*df["bid3vol"]
df["bid4sum"]=df["bid4"]*df["bid4vol"]
df["bid5sum"]=df["bid5"]*df["bid5vol"]
df["ask1sum"]=df["ask1"]*df["ask1vol"]
df["ask2sum"]=df["ask2"]*df["ask2vol"]
df["ask3sum"]=df["ask3"]*df["ask3vol"]
df["ask4sum"]=df["ask4"]*df["ask4vol"]
df["ask5sum"]=df["ask5"]*df["ask5vol"]
df["extra1"]=df["opened_position_qty "]*df["closed_position_qty"]
df["extra2"]=df["opened_position_qty "]*df["transacted_qty"]
df["extra3"]=df["transacted_qty"]*df["closed_position_qty"]
df["extra4"]=df["d_open_interest"]*df["opened_position_qty "]
df["extra5"]=df["d_open_interest"]*df["transacted_qty"]
df["extra6"]=df["d_open_interest"]*df["closed_position_qty"]
'''
df["bid1sq"]=df["bid1"]*df["bid1"]
df["bid2sq"]=df["bid2"]*df["bid2"]
df["bid3sq"]=df["bid3"]*df["bid3"]
df["bid4sq"]=df["bid4"]*df["bid4"]
df["bid5sq"]=df["bid5"]*df["bid5"]
df["ask1sq"]=df["ask1"]*df["ask1"]
df["ask2sq"]=df["ask2"]*df["ask2"]
df["ask3sq"]=df["ask3"]*df["ask3"]
df["ask4sq"]=df["ask4"]*df["ask4"]
df["ask5sq"]=df["ask5"]*df["ask5"]
df["bid1volsq"]=df["bid1vol"]*df["bid1vol"]
df["bid2volsq"]=df["bid2vol"]*df["bid2vol"]
df["bid3volsq"]=df["bid3vol"]*df["bid3vol"]
df["bid4volsq"]=df["bid4vol"]*df["bid4vol"]
df["bid5volsq"]=df["bid5vol"]*df["bid5vol"]
df["ask1volsq"]=df["ask1vol"]*df["ask1vol"]
df["ask2volsq"]=df["ask2vol"]*df["ask2vol"]
df["ask3volsq"]=df["ask3vol"]*df["ask3vol"]
df["ask4volsq"]=df["ask4vol"]*df["ask4vol"]
df["ask5volsq"]=df["ask5vol"]*df["ask5vol"]
'''

df=df.drop(["bid_vol_cols_min","ask2","bid1","ask4","bid5","ask3","bid_cols_min","last_price"],axis=1)

In [ ]:
df["date"] = pd.to_datetime("1.1.2019")
df["date"] = df["date"] + pd.to_timedelta(df["id"]/2,unit="s") # 500 ms per row

df["date"].describe()

# Split back into train and test, and build model

In [ ]:
all_train = df.loc[~df.y.isna()]
train = all_train.sample(frac=0.7, random_state=1)
vali = all_train.drop(train.index)
print(f"train shape {train.shape[0]}")
print(f"vali shape {vali.shape[0]}")
test = df.loc[df.y.isna()]
print(f"test shape {test.shape[0]}")

In [ ]:
train.drop(["id"],axis=1).to_csv("train_hft.csv.gz",index=False,compression="gzip")
vali.drop(["id"],axis=1).to_csv("vali_hft.csv.gz",index=False,compression="gzip")
test.to_csv("test_hft_nodates.csv.gz",index=False,compression="gzip")

In [ ]:
# we don't know if the test set has a temporal split, so we'll just try a random split for now
X = train.drop(["id","date","y"],axis=1)
y = train["y"]
X_vali = vali.drop(["id","date","y"],axis=1)
y_vali = vali["y"]

In [ ]:
train_pool = Pool(data=X,label = y)
vali_pool = Pool(data=X_vali,label = y_vali)

In [ ]:
'''
### hyperparameter tuning example grid for catboost : 
grid = {'learning_rate': [0.05, 0.1],
        'depth': [6, 11],
         'l2_leaf_reg': [1, 3,9],
        "iterations": [30],
        "custom_metric":['Logloss', 'AUC']}

model = CatBoostClassifier()

## can also do randomized search - more efficient typically, especially for large search space - `randomized_search`
grid_search_result = model.grid_search(grid, 
                                        train_pool,
                                        plot=True,
                                       refit = True, #  refit best model on all data
                                       partition_random_seed=42)

print(model.get_best_score())
'''

In [ ]:
model = CatBoostClassifier(learning_rate=0.1,depth=11,l2_leaf_reg=9,iterations=500)

#model.fit(train_pool, plot=True,silent=True)
model.fit(X=X,y=y, eval_set=vali_pool, plot=True,silent=True)
print(model.get_best_score())


## Features importances


In [ ]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    if score > 0.2:
        print('{0}: {1:.2f}'.format(name, score))

In [ ]:
import shap
#shap.initjs()

#explainer = shap.TreeExplainer(model)
#shap_values = explainer.shap_values(train_pool)

#visualize the training set predictions
#SHAP plots for all the data is very slow, so we'll only do it for a sample. Taking the head instead of a random sample is dangerous! 
#shap.force_plot(explainer.expected_value,shap_values[0,:300], X.iloc[0,:300])

In [ ]:
# summarize the effects of all the features
#shap.summary_plot(shap_values, X)

In [ ]:
## todo : PDP features +- from shap


## export predictions

In [ ]:
test["Predicted"] = model.predict(test.drop(["id","date","y",],axis=1),prediction_type='Probability')[:,1]
test[["id","Predicted"]].to_csv("submission6.csv",index=False)